In [20]:
from transformers import DistilBertTokenizer, DistilBertModel
import torch
import torch.nn.functional as F
from NPEET.npeet.entropy_estimators import mi

In [2]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased')

model = DistilBertModel.from_pretrained('distilbert-base-cased')

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

In [3]:
# Sample text
text = "Hello, how are you doing today?"

# Tokenize the text
inputs = tokenizer(text, return_tensors="pt")  # 'pt' stands for PyTorch tensors

print(inputs)

{'input_ids': tensor([[ 101, 8667,  117, 1293, 1132, 1128, 1833, 2052,  136,  102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [31]:
def get_embedding(overview):
    # Tokenize the input text with truncation and padding (if processing batches)
    token_inputs = tokenizer(overview, return_tensors='pt', truncation=True, padding=True)
    
    # Run the model in inference mode
    with torch.no_grad():
        outputs = model(**token_inputs)
    
    # Extract the last hidden state (shape: [batch_size, sequence_length, hidden_size])
    hidden_states = outputs.last_hidden_state
    
    # Apply mean pooling across the sequence dimension to get a single vector per input
    sentence_embedding = hidden_states.mean(dim=1)
    normalized_embedding = F.normalize(sentence_embedding, p=2, dim=1)
    return normalized_embedding
    

In [36]:
texts = [
    "Hello, world!",
    "How are you doing today?",
    "Transformers are amazing!"
]
toy_story = "Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences."

toy_story_2 = "Andy heads off to Cowboy Camp, leaving his toys to their own devices. Things shift into high gear when an obsessive toy collector named Al McWhiggen, owner of Al's Toy Barn kidnaps Woody. Andy's toys mount a daring rescue mission, Buzz Lightyear meets his match and Woody has to decide where he and his heart truly belong."

forrest_gump = "A man with a low IQ has accomplished great things in his life and been present during significant historic events - in each case, far exceeding what anyone imagined he could do. Yet, despite all the things he has attained, his one true love eludes him. 'Forrest Gump' is the story of a man who rose above his challenges, and who proved that determination, courage, and love are more important than ability."

forrest_gump_2= "The history of the United States from the 1950s to the '70s unfolds from the perspective of an Alabama man with an IQ of 75, who yearns to be reunited with his childhood sweetheart."

mission_impossible = "Ethan Hunt (Cruise) and his team are racing against time to track down a dangerous terrorist named Hendricks (Nyqvist), who has gained access to Russian nuclear launch codes and is planning a strike on the United States. An attempt to stop him ends in an explosion causing severe destruction to the Kremlin and the IMF to be implicated in the bombing, forcing the President to disavow them. No longer being aided by the government, Ethan and his team chase Hendricks around the globe, although they might still be too late to stop a disaster."

james_bond = "Quantum of Solace continues the adventures of James Bond after Casino Royale. Betrayed by Vesper, the woman he loved, 007 fights the urge to make his latest mission personal. Pursuing his determination to uncover the truth, Bond and M interrogate Mr. White, who reveals that the organization that blackmailed Vesper is far more complex and dangerous than anyone had imagined."

schindlers_list = "The true story of how businessman Oskar Schindler saved over a thousand Jewish lives from the Nazis while they worked as slaves in his factory during World War II"

bling_ring = "Inspired by actual events, a group of fame-obsessed teenagers use the Internet to track celebrities' whereabouts in order to rob their homes."
x = forrest_gump
y= forrest_gump_2
cos_sim = F.cosine_similarity(get_embedding(x), get_embedding(y))


tensor([0.9276])
0.6788200851375967


In [50]:

movie_texts = [
    "Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.",
    "A man with a low IQ has accomplished great things in his life and been present during significant historic events - in each case, far exceeding what anyone imagined he could do. Yet, despite all the things he has attained, his one true love eludes him. 'Forrest Gump' is the story of a man who rose above his challenges, and who proved that determination, courage, and love are more important than ability.",
    "Quantum of Solace continues the adventures of James Bond after Casino Royale. Betrayed by Vesper, the woman he loved, 007 fights the urge to make his latest mission personal. Pursuing his determination to uncover the truth, Bond and M interrogate Mr. White, who reveals that the organization that blackmailed Vesper is far more complex and dangerous than anyone had imagined.",
    "Inspired by actual events, a group of fame-obsessed teenagers use the Internet to track celebrities' whereabouts in order to rob their homes.",
    "Andy heads off to Cowboy Camp, leaving his toys to their own devices. Things shift into high gear when an obsessive toy collector named Al McWhiggen, owner of Al's Toy Barn kidnaps Woody. Andy's toys mount a daring rescue mission, Buzz Lightyear meets his match and Woody has to decide where he and his heart truly belong.",
    "Ethan Hunt (Cruise) and his team are racing against time to track down a dangerous terrorist named Hendricks (Nyqvist), who has gained access to Russian nuclear launch codes and is planning a strike on the United States. An attempt to stop him ends in an explosion causing severe destruction to the Kremlin and the IMF to be implicated in the bombing, forcing the President to disavow them. No longer being aided by the government, Ethan and his team chase Hendricks around the globe, although they might still be too late to stop a disaster."
    
]

# Tokenize the list of texts; the tokenizer will pad and truncate as needed
inputs = tokenizer(movie_texts, return_tensors="pt", padding=True, truncation=True)

# Run the model in inference mode to get embeddings
with torch.no_grad():
    outputs = model(**inputs)

# Obtain the token-level embeddings from the last hidden state
hidden_states = outputs.last_hidden_state  # Shape: [batch_size, seq_length, hidden_size]

# Apply mean pooling over the sequence dimension to get one embedding per movie
embeddings = hidden_states.mean(dim=1)  # Shape: [batch_size, hidden_size]

# Normalize each embedding using L2 normalization so that length does not affect similarity
norm_embeddings = F.normalize(embeddings, p=2, dim=1)

# Calculate the pairwise cosine similarity matrix using matrix multiplication
# Since the embeddings are normalized, the dot product equals cosine similarity
similarity_matrix = norm_embeddings @ norm_embeddings.T

# Print the resulting cosine similarity matrix
print("Cosine Similarity Matrix:")
print(similarity_matrix)


Cosine Similarity Matrix:
tensor([[1.0000, 0.8921, 0.9431, 0.9040, 0.9769, 0.9102],
        [0.8921, 1.0000, 0.9230, 0.8843, 0.9060, 0.9052],
        [0.9431, 0.9230, 1.0000, 0.9154, 0.9604, 0.9568],
        [0.9040, 0.8843, 0.9154, 1.0000, 0.9227, 0.8918],
        [0.9769, 0.9060, 0.9604, 0.9227, 1.0000, 0.9302],
        [0.9102, 0.9052, 0.9568, 0.8918, 0.9302, 1.0000]])


In [43]:
import numpy as np


tensor(0)


In [55]:
def second_highest_index(lst):
    if len(lst) < 2:
        raise ValueError("List must have at least two elements.")
    
    highest_idx = 0
    second_highest_idx = None
    
    for i in range(1, len(lst)):
        if lst[i] > lst[highest_idx]:
            second_highest_idx = highest_idx
            highest_idx = i
        elif lst[i] != lst[highest_idx]:
            # Update second highest index if it's either unset or the current value is larger
            if second_highest_idx is None or lst[i] > lst[second_highest_idx]:
                second_highest_idx = i
                
    if second_highest_idx is None:
        raise ValueError("All elements are equal; no distinct second highest element.")
    
    return second_highest_idx

def get_sorted_indices(lst):
    """
    Returns a list of indices sorted by their corresponding values in descending order.
    
    Example:
      lst = [2, 7, 3, 10, 5]
      get_sorted_indices_desc(lst) -> [3, 1, 4, 2, 0]
    """
    return sorted(range(len(lst)), key=lambda i: lst[i], reverse=True)





In [58]:
given_movie_id= 5
similar_movie_id = get_sorted_indices(similarity_matrix[given_movie_id])
print(similar_movie_id)

[5, 2, 4, 0, 1, 3]
